# B-ALL Classification - Model Inference

This notebook demonstrates how to load and use the trained DenseNet-121 model for B-cell Acute Lymphoblastic Leukemia (B-ALL) classification.

## Overview

The inference pipeline includes:
1. Loading the pre-trained model weights
2. Preprocessing new images with CLAHE enhancement (same as training)
3. Making predictions on blood cell images
4. Interpreting results with confidence scores

**Author:** Shimaa Abu Youcef

In [ ]:
import torch
import torch.nn as nn
from torchvision import models, transforms
from PIL import Image
import cv2
import numpy as np

## 1. Import Required Libraries

First, we import the necessary libraries for model loading, image processing, and inference.

In [ ]:
# --- 1. Load the saved model ---
class_names = ["Benign", "Early Pre-B", "Pre-B", "Pro-B"]

loaded_model = models.densenet121(pretrained=False)
num_ftrs = loaded_model.classifier.in_features
loaded_model.classifier = nn.Linear(num_ftrs, len(class_names))

# Load weights
model_path = "leukeai_densenet121.pth"
loaded_model.load_state_dict(torch.load(model_path, map_location=torch.device("cpu")))
loaded_model.eval()

print("✅ Model loaded successfully!")

## 2. Load the Trained Model

Here we load the pre-trained DenseNet-121 model that was saved during training. The model expects the saved weights file `leukeai_densenet121.pth` to be in the same directory.

**Classes:**
- **Benign**: Normal, healthy blood cells
- **Early Pre-B**: Early stage B-cell acute lymphoblastic leukemia
- **Pre-B**: Pre-B cell acute lymphoblastic leukemia
- **Pro-B**: Pro-B cell acute lymphoblastic leukemia

In [ ]:
# --- 2. Define preprocessing (same as training) -> check our firs notebook 'main.ipynb'
IMG_SIZE = 224
test_transforms = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

def preprocess_image(img_path):
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Apply CLAHE preprocessing (same as training)
    lab = cv2.cvtColor(img, cv2.COLOR_RGB2LAB)
    l, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    cl = clahe.apply(l)
    limg = cv2.merge((cl,a,b))
    img = cv2.cvtColor(limg, cv2.COLOR_LAB2RGB)

    pil_img = Image.fromarray(img)
    tensor = test_transforms(pil_img).unsqueeze(0) 
    return tensor


## 4. Making Predictions

To use this model for inference on new blood cell images:

```python
# Example usage:
image_path = "path/to/your/blood_cell_image.jpg"

# Preprocess the image
input_tensor = preprocess_image(image_path)

# Make prediction
with torch.no_grad():
    outputs = loaded_model(input_tensor)
    probabilities = torch.nn.functional.softmax(outputs[0], dim=0)
    predicted_class = torch.argmax(probabilities).item()
    confidence = probabilities[predicted_class].item()

# Display results
print(f"Predicted class: {class_names[predicted_class]}")
print(f"Confidence: {confidence:.4f}")

# Show all class probabilities
for i, prob in enumerate(probabilities):
    print(f"{class_names[i]}: {prob:.4f}")
```

**Note:** Make sure your input images are microscopy images of blood smears for accurate classification.

## 3. Image Preprocessing

The preprocessing pipeline must match exactly what was used during training:

1. **CLAHE Enhancement**: Improves contrast using Contrast Limited Adaptive Histogram Equalization in LAB color space
2. **Resize**: Images are resized to 224×224 pixels (DenseNet-121 input size)
3. **Normalization**: Uses ImageNet statistics for transfer learning compatibility

This preprocessing function takes an image path and returns a tensor ready for model inference.